# Install Repository

In [ ]:
#first time run
!git clone https://github.com/antonioverdi/MLReproChallenge.git
#At the moment all requirements are met by Google Colab already I believe
import os
os.chdir("MLReprochallenge")
!pip install -r requirements.txt

In [ ]:
#subsequent runs
!git pull
!pip install -r requirements.txt

# Train Models
### Main.py Arguments
<table>
    <tr><th>Argument</th><th>Default</th><th>help</th></tr>
    <tr><td>--arch, - a</td><td>str: 'resnet56'</td><td>model architecture</td></tr>
    <tr><td>-j, --workers</td><td>int: 4</td><td>number of data loading workers</td></tr>
    <tr><td>--epochs</td><td>int: 182</td><td>number of total epochs to run</td></tr>
    <tr><td>--start-epoch</td><td>int: 0</td><td>manual epoch number, for restarts</td></tr>
    <tr><td>-b, --batch-size</td><td>int: 128</td><td>mini-batch size</td></tr>
    <tr><td>--lr, --learning-rate</td><td>float: 0.1</td><td>initial learning rate</td></tr>
    <tr><td>--momentum</td><td>float: 0.9</td><td>momentum</td></tr>
    <tr><td>--weight-decay, --wd</td><td>float: 2e-4</td><td>weight decay</td></tr>
    <tr><td>--print-freq, -p</td><td>int: 50</td><td>print frequency</td></tr>
    <tr><td>--resume</td><td>str: ' '</td><td>path to latest checkpoint</td></tr>
    <tr><td>-e, --evaluate</td><td>bool: False</td><td>evaluate model on validation set</td></tr>
    <tr><td>--pretrained</td><td>bool: False</td><td>use pre-trained model</td></tr>
    <tr><td>--half</td><td>bool: False</td><td>use half-precision float(16-bit)</td></tr>
    <tr><td>--save-dir</td><td>str: 'save_temp'</td><td>Directory used to save trained models</td></tr>
    <tr><td>--save-every</td><td>int: 10</td><td>Save checkpoint at every specified number of epochs</td></tr>
    <tr><td>--colab</td><td>bool: False</td><td>Set this to true when running in Google Colab</td></tr>
   </table>

In [ ]:
%%time
!python main.py --colab

# Test Models

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import types

In [20]:
X = torch.rand((10,5))
y = torch.ones(10, dtype=torch.long)
y[:5] = 0

In [24]:
class TestModel(nn.Module):
    def __init__(self, init_strategy="kaiming"):
        super(TestModel, self).__init__()
        self.layer1 = nn.Linear(5,3)
        self.layer2 = nn.Linear(3,5)
        if init_strategy == "kaiming":
            self.apply(_kaiming_weights_init)
        elif init_strategy == "xavier":
            self.apply(_xavier_weights_init)
            
    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = self.layer2(out)
        return out

In [25]:
def _kaiming_weights_init(m):
    classname = m.__class__.__name__
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)
        
def _xavier_weights_init(m):
    classname = m.__class__.__name__
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.xavier_normal_(m.weight)

def snip_forward_conv2d(self, x):
        return F.conv2d(x, self.weight * self.weight_mask, self.bias,
                        self.stride, self.padding, self.dilation, self.groups)

def snip_forward_linear(self, x):
        return F.linear(x, self.weight * self.weight_mask, self.bias)

In [36]:
def snip_mask(model, batch, labels, compression):
    
    
    for layer in model.modules():
        #create pruning masks manually
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            layer.weight_mask = nn.Parameter(torch.ones_like(layer.weight)) 
#             layer.weight.requires_grad = False #computing gradient of mask not weights

        #monkey-patch forward methods
        if isinstance(layer, nn.Conv2d):
            layer.forward = types.MethodType(snip_forward_conv2d, layer)
            
        if isinstance(layer, nn.Linear):
            layer.forward = types.MethodType(snip_forward_linear, layer)
            
    #compute gradients of weight_mask (connections)
    model.zero_grad()
    out = model.forward(batch)
    loss = F.nll_loss(out, labels)
    loss.backward()
    
    absolute_saliency = []
    for layer in model.modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            absolute_saliency.append(torch.abs(layer.weight_mask.grad))
            
    saliency_scores = torch.cat([torch.flatten(x) for x in absolute_saliency])
    denominator = torch.sum(saliency_scores)
    saliency_scores.div_(denominator)
    
    kappa = int(len(saliency_scores) * compression)
    sorted_scores, indices = torch.topk(saliency_scores, kappa, sorted=True)
    threshold = sorted_scores[-1]
    
    connection_masks = []
    for c in absolute_saliency:
        connection_masks.append(((c / denominator) >= threshold).float())
    
    return connection_masks

def apply_snip(model, connection_masks):
    prunable_layers = filter(lambda layer: isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear), model.modules())

    for layer, mask in zip(prunable_layers, connection_masks):
#         assert (layer.weight.shape == keep_mask.shape)

        def hook_factory(keep_mask):
            """
            The hook function can't be defined directly here because of Python's
            late binding which would result in all hooks getting the very last
            mask! Getting it through another function forces early binding.
            source: https://github.com/mil-ad/snip/blob/master/train.py
            """

            def hook(grads):
                return grads * keep_mask

            return hook

        # Set the masked weights to zero (biases are ignored)
        layer.weight.data[mask == 0.] = 0.
        # Make sure their gradients remain zero. Register_hook gets called whenever a gradient is collected
        layer.weight.register_hook(hook_factory(mask))

In [37]:
model = TestModel(init_strategy="xavier")
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
masks = snip_mask(model, X, y, 0.50)
apply_snip(model, masks)


In [39]:
model1 = TestModel(init_strategy="xavier")
optimizer = torch.optim.SGD(model1.parameters(), lr=0.0001)
output = model1.forward(X)
print(output)
masks = snip_mask(model1, X, y, 0.5)
apply_snip(model1, masks)
output = model1.forward(X)
print(output)

tensor([[-0.2113, -0.6545, -0.6346,  0.2245,  0.0700],
        [-0.1113, -0.5254, -0.5355,  0.2784,  0.0494],
        [-0.1040, -0.6149, -0.7418,  0.0143, -0.0199],
        [-0.1213, -0.5356, -0.5395,  0.2805,  0.0533],
        [-0.1682, -0.6183, -0.6340,  0.1948,  0.0477],
        [-0.6083, -1.2680, -1.2461, -0.2632,  0.0826],
        [ 0.0513, -0.3832, -0.5205,  0.1825, -0.0305],
        [ 0.1009, -0.3099, -0.4514,  0.2343, -0.0344],
        [ 0.1009, -0.3099, -0.4514,  0.2343, -0.0344],
        [-0.0264, -0.4392, -0.5018,  0.2607,  0.0159]],
       grad_fn=<AddmmBackward>)
tensor([[-0.2113, -0.6545, -0.6346,  0.2245,  0.0700],
        [-0.1113, -0.5254, -0.5355,  0.2784,  0.0494],
        [-0.1040, -0.6149, -0.7418,  0.0143, -0.0199],
        [-0.1213, -0.5356, -0.5395,  0.2805,  0.0533],
        [-0.1682, -0.6183, -0.6340,  0.1948,  0.0477],
        [-0.6083, -1.2680, -1.2461, -0.2632,  0.0826],
        [ 0.0513, -0.3832, -0.5205,  0.1825, -0.0305],
        [ 0.1009, -0.3099, -0.45

In [41]:
for layer in model.parameters():
    print(layer)

Parameter containing:
tensor([[ 0.0000, -0.1337,  0.3056,  0.2123,  0.4977],
        [ 0.0000,  0.0000,  0.0000,  0.0000, -0.5045],
        [ 0.5046,  0.3541,  0.0000,  0.3363,  1.2411]], requires_grad=True)
Parameter containing:
tensor([ 0.3632,  0.1242, -0.4400], requires_grad=True)
Parameter containing:
tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]], requires_grad=True)
Parameter containing:
tensor([[ 0.9418,  0.3743, -0.1923],
        [ 0.2898, -0.5059, -0.1182],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], requires_grad=True)
Parameter containing:
tensor([ 0.3812,  0.2165,  0.3906,  0.4295, -0.1825], requires_grad=True)
Parameter containing:
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)


In [18]:
print(X)
print(y)

tensor([[0.8507, 0.8685, 0.7784, 0.6547, 0.7529],
        [0.0561, 0.8708, 0.2868, 0.2654, 0.6038],
        [0.0824, 0.7327, 0.5353, 0.8400, 0.6228],
        [0.2959, 0.5033, 0.7964, 0.8578, 0.9757],
        [0.0246, 0.0190, 0.2140, 0.9614, 0.1417],
        [0.8252, 0.5460, 0.0270, 0.6644, 0.5456],
        [0.5410, 0.0719, 0.3772, 0.5851, 0.8779],
        [0.2785, 0.2152, 0.5592, 0.9359, 0.2258],
        [0.2346, 0.0415, 0.2520, 0.1155, 0.2644],
        [0.1498, 0.0111, 0.7135, 0.9518, 0.1588]])
tensor([0., 0., 0., 0., 0., 1., 1., 1., 1., 1.])
